<a href="https://colab.research.google.com/github/Sweta-Das/LangChain-HuggingFace-LLM/blob/SentenceTransformers/PDF_Text_Embedding%26Querying_CosineSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# %%capture
%pip -q install PyPDF2 pdfplumber langchain sentence-transformers transformers numba

In [17]:
%pip -q install llama-cpp-python

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
from langchain.llms import LlamaCpp
from langchain import HuggingFaceHub
from PyPDF2 import PdfReader
from numba import jit, cuda
from pdfplumber import pdf
import numpy as np
import sys, random
import torch
import time
import os

**About Libraries**:<br>
- *RecursiveCharacterTextSplitter* : a function to split text into smaller chunks based on a specified character set & chunk size. Recursive splitting works by repeatedly splitting the text into smaller pieces until it reaches a desired size or encounters a separator character.
- *SentenceTransformer* : class used for embedding sentences into numerical vectors for various NLP tasks
- *LLMChain* : a LangChain's class specifically designed to interact with LLMs
- *HuggingFaceHub* :  class that joins LangChain with Hugging Face
- *PyPDF2* : a library that works with PDF files in Python; *PdfReader* reads the PDF docs' content
- *pdfplumber* : a library for extracting text & data from PDF docs; *pdf* works with PDFs
- *numba* : a library in Python ecosystem used for high-performance numerical computing. It provides **JIT (Just In Time)** compiler *(@jit)* that translates Python functions into optimized machine code at runtime. It also support **cuda** like *(@cuda.jit)* to execute code on NVIDIA GPUs.



In [19]:
# Accessing through HuggingFace Access Token
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HUGGINGFACEHUB_API_TOKEN'

In [20]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive/')
model = 'drive/MyDrive/LLM_Model/mistral-7b-instruct-v0.1.Q3_K_S.gguf'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [21]:
def progressBar(count_value, total, suffix=''):
  # Designing progress bar (==---)
  bar_length = 100
  filled_up_length = int(round(bar_length * count_value / float(total)))
  percent = round(100.0 * count_value/float(total), 1)
  bar = '=' * filled_up_length + '-' * (bar_length - filled_up_length)
  sys.stdout.write('[%s] %s%s ...%s\r' %(bar, percent, '%', suffix))
  sys.stdout.flush()

### Reading the pdf file

In [40]:
def load_split_pdf(pdf_path):
  # Reading pdf in binary mode
  pdf_loader = PdfReader(open(pdf_path, "rb"))
  pdf_text = ""

  # Reading only 8 pages of pdf
  for page_num in range(min(8, len(pdf_loader.pages))): # len(pdf_loader.pages)
    # Loading page
    pdf_page = pdf_loader.pages[page_num]
    # Extracting text
    pdf_text += pdf_page.extract_text()

  last_page_num = len(pdf_loader.pages) - 1
  if last_page_num >= 0:
    last_page = pdf_loader.pages[last_page_num]
    pdf_text += last_page.extract_text()
  progressBar(2, 7)
  return pdf_text

### Recursive Text Character Splitter

In [41]:
def split_text_using_RCTS(pdf_text):

  # Splitting text recursively
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 2048,
      chunk_overlap = 64
  )
  split_texts = text_splitter.split_text(pdf_text)

  # Separating texts at paragraphs
  paragraphs = []
  for text in split_texts:
    paragraphs.extend(text.split('\n'))

  progressBar(3, 7)
  return paragraphs

### Sentence Transformer

In [42]:
# Initializing sentence transformer
def Initialize_sentence_transformer():
  model_name = "sentence-transformers/all-MiniLM-L6-v2"
  embeddings = SentenceTransformer(model_name)

  progressBar(4, 7)
  return embeddings

In [43]:
# Encoding each paragraph
def encode_each_paragraph(paragraphs, embeddings):
  responses = []
  for paragraph in paragraphs:
    response = embeddings.encode([paragraph], convert_to_tensor=True)
    responses.append((paragraph, response))

  progressBar(5, 7)
  return responses

In [47]:
# Choosing most relevant sentence
def choose_most_relevant_sentence(embeddings, responses, query):
  query_embedding = embeddings.encode([query], convert_to_tensor=True)
  best_response = None
  best_similarity = -1.0
  answers = []

  for paragraph, response in responses:
    # Finding cosine similarity between query embedding and response
    similarity = util.pytorch_cos_sim(query_embedding, response).item()

    if similarity >= 0.8:
      # count += 1
      answers.append(paragraph)

  answer = "\n".join(answers)

  progressBar(6, 7)
  return answer

### Querying the LLM

In [49]:
def get_query():
    query = input("Enter your question\n")
    progressBar(1, 7)
    return query

In [50]:
def query_the_llm(answer, llm_model, query):
    prompt_message = answer + "\n" + query

    final_response = llm_model.generate(prompts=[prompt_message])

    return final_response

In [52]:
# Loading the LLM Model
def main():
  start_time = time.time()
  pdf_path = "./HandbookOfTechnicalAnalysis.pdf"
  pdf_text = load_split_pdf(pdf_path)
  paragraphs = split_text_using_RCTS(pdf_text)
  embeddings = Initialize_sentence_transformer()
  responses = encode_each_paragraph(paragraphs=paragraphs, embeddings=embeddings)
  # print(responses)
  query = get_query()
  answer = choose_most_relevant_sentence(embeddings=embeddings, responses=responses, query=query)

  llm = LlamaCpp(
      streaming = True,
      model_path = "/content/drive/MyDrive/LLM_Model/mistral-7b-instruct-v0.1.Q3_K_S.gguf",
      temperature = 0.75, # degree of randomness
      top_p = 1,
      verbose = True,
      n_ctx = 4096 # max no. of tokens to generate
  )

  final_response = query_the_llm(answer=answer, llm_model=llm, query=query)

  print ("The answer from model is\n", final_response)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print(f"Execution time: {elapsed_time/60} minutes \n")

  progressBar(7, 7)

if __name__ == "__main__":
  main()

Enter your question
How many users does Elearnmarkets company have?


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/LLM_Model/mistral-7b-instruct-v0.1.Q3_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - k

The answer from model is
 generations=[[Generation(text='\n\n## Answer (6)\n\nAs of my knowledge up to 2021, elearnmarkets has 3,589 registered users as of July 2018.\n\nThis information can be found on the following webpage: https://elearnmarkets.com/about-us/\n\nComment: This answer is correct for July 2018 but is incomplete. Please update it to reflect the current state of affairs.')]] llm_output=None run=[RunInfo(run_id=UUID('c05a3cee-268e-4c03-9629-e5dedad13ab8'))]
Execution time: 1.8362728794415792 minutes 



### Referenced From: <br>
[**Querying a PDF file using LLM models and Sentence transformer**](https://medium.com/@yashashm77/querying-a-pdf-file-using-llm-models-and-sentence-transformer-b3d4d0b40f7d)<br>